## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,WeatherPy_Database,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Arraial Do Cabo,BR,2021-03-15 00:57:26,-22.9661,-42.0278,77.13,84,100,15.86
1,1,Victoria,HK,2021-03-15 00:56:28,22.2855,114.1577,71.01,78,96,1.99
2,2,Srednekolymsk,RU,2021-03-15 00:57:34,67.4500,153.6833,-8.50,90,0,8.77
3,3,Ballybofey,IE,2021-03-15 00:57:34,54.8000,-7.7833,43.39,95,99,7.47
4,4,Hithadhoo,MV,2021-03-15 00:57:35,-0.6000,73.0833,84.06,63,100,5.99


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 68
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,WeatherPy_Database,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Arraial Do Cabo,BR,2021-03-15 00:57:26,-22.9661,-42.0278,77.13,84,100,15.86
1,1,Victoria,HK,2021-03-15 00:56:28,22.2855,114.1577,71.01,78,96,1.99
7,7,Imbituba,BR,2021-03-15 00:57:36,-28.2400,-48.6703,73.40,94,75,3.44
9,9,Puerto Ayora,EC,2021-03-15 00:57:36,-0.7393,-90.3518,79.00,90,100,4.23
15,15,Hilo,US,2021-03-15 00:53:06,19.7297,-155.0900,75.00,78,90,6.91
16,16,Bangui,CF,2021-03-15 00:57:38,4.3612,18.5550,73.83,88,100,2.35
20,20,Hambantota,LK,2021-03-15 00:57:40,6.1241,81.1185,71.60,100,20,2.30
24,24,Port Elizabeth,ZA,2021-03-15 00:57:06,-33.9180,25.5701,69.80,94,90,5.75
26,26,Lahaina,US,2021-03-15 00:57:41,20.8783,-156.6825,78.80,100,75,10.36
29,29,Pisco,PE,2021-03-15 00:57:09,-13.7000,-76.2167,78.80,61,0,17.27


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

WeatherPy_Database    208
City                  208
Country               207
Date                  208
Lat                   208
Lng                   208
Max Temp              208
Humidity              208
Cloudiness            208
Wind Speed            208
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,77.13,-22.9661,-42.0278,
1,Victoria,HK,71.01,22.2855,114.1577,
7,Imbituba,BR,73.40,-28.2400,-48.6703,
9,Puerto Ayora,EC,79.00,-0.7393,-90.3518,
15,Hilo,US,75.00,19.7297,-155.0900,
16,Bangui,CF,73.83,4.3612,18.5550,
20,Hambantota,LK,71.60,6.1241,81.1185,
24,Port Elizabeth,ZA,69.80,-33.9180,25.5701,
26,Lahaina,US,78.80,20.8783,-156.6825,
29,Pisco,PE,78.80,-13.7000,-76.2167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 20000,
    "type": "lodging",
    "key": g_key
    
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,77.13,-22.9661,-42.0278,Pousada Porto Praia
1,Victoria,HK,71.01,22.2855,114.1577,Mini Hotel Central
7,Imbituba,BR,73.40,-28.2400,-48.6703,Praia Hotel Imbituba
9,Puerto Ayora,EC,79.00,-0.7393,-90.3518,Finch Bay Galapagos Hotel
15,Hilo,US,75.00,19.7297,-155.0900,Hilo Hawaiian Hotel


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Hotel_data.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))